In [55]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm
import sklearn

**Explore data distribution and remove outliers**

In [ ]:
# -- Read matrix containing embeddings and labels
path_X = '/Users/leandrojorqueravalero/Desktop/PhD/synthetic_ORF2/data/X.csv' 
X = pd.read_csv(path_X)

In [ ]:
# Remove undesired column and filter mammalian entries
X = X.drop(columns=['Unnamed: 0'])
X_mammals = X[X['Taxonomy'] == 'Mammalia']

**Check how CN and prot_len distribution changes when removing non-mammals**

In [ ]:
X['Non_redundant'].hist(bins = 20, figsize=(12,8))

In [ ]:
X_mammals['Non_redundant'].hist(bins = 20, figsize=(12,8))

In [ ]:
X['prot_len'].hist(bins = 20, figsize=(12,8))

In [ ]:
X_mammals['prot_len'].hist(bins = 20, figsize=(12,8))

**Explore correlation between between each vector dimension and CN values**

In [56]:
corr_matrix = X_mammals.corr() # generate correlation matrix
correlations = corr_matrix['Non_redundant'].sort_values(ascending=False) # check columns (dimensions) that correlates more to CN

In [67]:
most_correlated = correlations.nlargest(30)

In [83]:
# This map explores pairwise correlation among all dimensions (not only against CN)
df = X_mammals.iloc[:10,:10]
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7,8
0,1.000000,0.811557,0.359086,-0.606246,0.805365,-0.334240,-0.422725,-0.868861,0.175155
1,0.811557,1.000000,-0.207301,-0.459636,0.765621,-0.198438,-0.704119,-0.701645,0.601115
2,0.359086,-0.207301,1.000000,-0.444365,0.179567,-0.366472,0.405945,-0.408797,-0.757826
3,-0.606246,-0.459636,-0.444365,1.000000,-0.572806,0.768669,0.401538,0.796974,-0.018429
4,0.805365,0.765621,0.179567,-0.572806,1.000000,-0.424966,-0.657584,-0.908491,0.297859
5,-0.334240,-0.198438,-0.366472,0.768669,-0.424966,1.000000,0.184201,0.614237,-0.060814
6,-0.422725,-0.704119,0.405945,0.401538,-0.657584,0.184201,1.000000,0.462398,-0.579601
7,-0.868861,-0.701645,-0.408797,0.796974,-0.908491,0.614237,0.462398,1.000000,-0.192758
8,0.175155,0.601115,-0.757826,-0.018429,0.297859,-0.060814,-0.579601,-0.192758,1.000000


In [112]:
corre_test = X_mammals.iloc[:10]  # select only 10 first rows for test
dimensions = corre_test.iloc[:,1:1025] # select vector dimensions
CN = corre_test['Non_redundant'] # select cn column

In [84]:
from sklearn import feature_selection

In [115]:
pearson = sklearn.feature_selection.r_regression(dimensions, CN, force_finite=True)

In [116]:
len(pearson)

1024

**Applying PCA over all PC**

In [ ]:
scale = StandardScaler()
pca = PCA() # Define PCA function

my_pca = pca.fit_transform(matched_emb)   # Scale data and apply PCA on embeddings df
dataframe_pca = pd.DataFrame(my_pca)

**Performing linear regression**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)  # split dataset